## Day 8

https://adventofcode.com/2021/day/8

In [49]:
def readInput8(filename):
    with open(filename) as f:
        return [ [ p.split() for p in l.strip('\n').split(" | ") ] for l in f.readlines() ]

i0 = readInput8("data/day08test01.txt")
i1 = readInput8("data/day08test02.txt")
i2 = readInput8("data/input08.txt")

### Part 1
Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits. Count only digits in the output values.

In [50]:
from collections import Counter

def countUnique(o):
    # 1 = 2S ; 4 = 4S ; 7 = 3S ; 8 = 7S
    c = Counter([ len(d) for d in o ])
    return c[2]+c[4]+c[3]+c[7]

def part1(i0):
    return sum([ countUnique(o) for i,o in i0 ])

print("Test 1:",part1(i1))
print("Part 1:",part1(i2))

Test 1: 26
Part 1: 512


### Part 2

Within an entry, the same wire/segment connections are used (but you don't know what the connections actually are).

Not trying to be smart, trying all wiring permutations for each line...

In [122]:
from itertools import permutations

display = {}
display["abcefg"]  = 0
display["cf"]      = 1
display["acdeg"]   = 2
display["acdfg"]   = 3
display["bcdf"]    = 4
display["abdfg"]   = 5
display["abdefg"]  = 6
display["acf"]     = 7
display["abcdefg"] = 8
display["abcdfg"]  = 9

def checkCode(di,do):
    return sum([1 if i in display.keys() else 0 for i in di ])==len(di) and \
           sum([1 if o in display.keys() else 0 for o in do ])==len(do)

def part2(i0,verbose=True):
    output = 0
    origin = 'abcdefg'
    for i,o in i0:
        # find correct wiring for current line
        di = ""
        do = ""
        for p in permutations(orig):
            scramb = "".join(p)
            # generate code for given permutation
            code = {}
            for ori,scr in zip(origin,scramb):
                code[scr] = ori
            # test code on line
            di = [ "".join(sorted([ code[s] for s in ii ])) for ii in i ]
            do = [ "".join(sorted([ code[s] for s in oo ])) for oo in o ]
            if checkCode(di,do):
                break
        # output value
        value = int("".join(str(display[oo]) for oo in do))
        if verbose: 
            for _o in o:
                print("{:7s} ".format(_o),end=" ")
            print(value)
        output += value
    return output
            
part2(i1)

fdgacbe  cefdb    cefbgd   gcbe     8394
fcgedb   cgb      dgebacf  gc       9781
cg       cg       fdcagb   cbg      1197
efabcd   cedba    gadfec   cb       9361
gecf     egdcabf  bgf      bfgea    4873
gebdcfa  ecba     ca       fadegcb  8418
cefg     dcbef    fcge     gbcadfe  4548
ed       bcgafe   cdgba    cbgef    1625
gbdfcae  bgc      cg       cgb      8717
fgae     cfgab    fg       bagce    4315


61229

In [113]:
part2(i2,False)

1091165